In [1]:
import time

time.sleep(60 * 60 * 8) # sleep 8 hours

In [2]:
import numpy as np
import torch
import os
from tqdm.notebook import trange, tqdm

from torchvision import transforms
from models.retinanet.dataloader import CocoDataset, Resizer, Normalizer
from models.retinanet import model, coco_eval

from src.coco_eval import prepare_dataset_folder 
from retinanet_utils import *


np.set_printoptions(precision=3, linewidth=1000)
torch.set_printoptions(precision=3, linewidth=1000)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

#### Load Model

In [3]:
retinanet = model.resnet50(num_classes=80, pretrained=True, model_dir='data/models').to(device)
retinanet.load_state_dict(torch.load('data/models/coco_resnet_50_map_0_335_state_dict.pt'))
retinanet.training = False
retinanet.freeze_bn()    # freeze batch-norm layers
retinanet.eval();

#### I-FGSM

In [4]:
from src.utils import selective_l1_loss_indexed as selective_l1_loss
from src.utils import l1_loss

def fgsm(image: torch.tensor, model: torch.nn.Module, steps: int = 5, epsilon: float = 0.1, 
        threshold: float = 0.5, loss_fn: str = 'selective', device: str = 'cuda'):
    # prep targets
    mask = torch.zeros_like(image, requires_grad=True, device=device)
    image.requires_grad = True
    criterion = selective_l1_loss if loss_fn == 'selective' else l1_loss

    for _ in range(steps):
        _, probs, _ = model.forward_raw(mask + image)
        loss = criterion(probs, threshold=threshold)
        model.zero_grad()
        loss.backward()

        gradient = mask.grad.data.detach()
        mask.data = mask.data - gradient.sign() * epsilon
        mask.grad.data.zero_()
        image.grad.data.zero_()

    # fix the changes
    cloacked_image = (image + mask).detach()

    return cloacked_image

In [5]:
%%script echo skipping
import time

images = []
iters = 5

for fname in os.listdir('data/coco_samples')[:6]:
    image, _ = read_image(f'data/coco_samples/{fname}')
    images.append(image_to_tensor(image))
    
for threshold in [0, 0.5]:
    for steps in [1, 5]:
        ti = time.time()

        for _ in range(iters):
            for img in images:
                fgsm(img, retinanet, steps=steps, threshold=threshold)
        
        method = '' if threshold == 0 else 'S-'
        attack = 'FGSM' if steps == 1 else 'PGD'
        et = (time.time() - ti) / len(images) / iters
        print(f'{method}{attack}: {et:.3f}s per img')

skipping


#### Image by Image Cloaking

In [6]:
%%script echo skipping
annotated_images = []
annotated_cloacked_images = []
differences = []

for fname in tqdm(os.listdir('data/coco_samples')[:6]):
    ''' Process vanilla image '''
    image, scale = read_image(f'data/coco_samples/{fname}')
    annotated_image = annotate_image(image, retinanet, confidence=0.5)

    ''' Cloak and re-process image'''
    cloacked_image = fgsm(image_to_tensor(image), retinanet, steps=10, epsilon=0.025, threshold=0.3)
    cloacked_image = image_to_numpy(cloacked_image)
    annotated_cloacked_image = annotate_image(cloacked_image, retinanet, confidence=0.5)
    difference = abs(image - cloacked_image)

    annotated_images.append(annotated_image)
    annotated_cloacked_images.append(annotated_cloacked_image)
    differences.append(difference * 10)
    # print(f'Using {torch.cuda.memory_allocated() / 1000 / 1000: .0f} MB in GPU')


show_images_sequence(annotated_images, annotated_cloacked_images, differences, nrows=1, ncols=6)

skipping


### COCO Attack

In [7]:
%%script echo skipping
# retinanet images are standardized. min(range) = 4.4
steps = 10
epsilon = 1 / 10
threshold = 0.5
method = 'fgsm' if steps == 1 else 'pgd'
loss_fn = 'selective' # 'l1'

src_fnames, dst_fnames = prepare_dataset_folder('data/coco', f'data/coco_cloaked/retinanet_{loss_fn}_{method}_{epsilon:.3f}')


for src, dst in tqdm(list(zip(src_fnames, dst_fnames))):
    image, dims = read_image(src)
    image = image_to_tensor(image)

    cloacked_image = fgsm(image, retinanet, steps, 4.4 * epsilon / steps, threshold, loss_fn)
    cloacked_image = image_to_numpy(cloacked_image)

    save_image(dst, cloacked_image, dims)

skipping


In [8]:
%%script echo skipping

for loss_fn in ['l1', 'selective']:
    for steps in [1, 5]:
        for epsilon in [1/10, 1/50]:
            threshold = 0.5
            method = 'fgsm' if steps == 1 else 'pgd'

            dst_folder = f'{loss_fn}_{method}_{epsilon:.3f}'
            src_fnames, dst_fnames = prepare_dataset_folder('data/coco', f'C:/Users/tcilloni/Desktop/coco_cloaked/retinanet_{dst_folder}')


            for src, dst in tqdm(list(zip(src_fnames, dst_fnames)), desc=dst_folder):
                image, dims = read_image(src)
                image = image_to_tensor(image)

                cloacked_image = fgsm(image, retinanet, steps, 4.4 * epsilon / steps, threshold, loss_fn)
                cloacked_image = image_to_numpy(cloacked_image)

                save_image(dst, cloacked_image, dims)

skipping


### COCO Evaluation - PyCOCO Tools

In [9]:
%%script echo skipping
attacker_model = ['ssd', 'retinanet', 'frcnn'][0]
attack_method = ['pgd', 'fgsm'][0]

dataset_dir = f'data/coco_cloaked/{attacker_model}_{attack_method}'
dataset_dir = 'coco_test'

dataset = CocoDataset(dataset_dir, set_name='val2017', transform=transforms.Compose([Normalizer(), Resizer()]))
coco_eval.evaluate_coco(dataset, retinanet)

skipping


### COCO Evaluation - FiftyOne

In [10]:
%%script echo skipping
from src.coco_eval import load_coco_dataset_in_fiftyone

dataset = load_coco_dataset_in_fiftyone('data/coco', use_cached=False, max_samples=100)
# session = fo.launch_app(dataset, desktop=True)  # comment me out if running the cell multiple times

skipping


In [11]:
%%script echo skipping
for sample in tqdm(dataset):
    detections = detections_fiftyone_format(sample.filepath, retinanet)
    sample['predictions'] = detections
    sample.save()

# session.view = dataset.view()
results = dataset.evaluate_detections('predictions', gt_field='detections', eval_key='eval', compute_mAP=True)
print(f'mAP: {results.mAP():.3f}')

skipping


### COCO Evaluation - Full

In [12]:
%%script echo skipping
from src.coco_eval import load_coco_dataset_in_fiftyone
from retinanet_utils import *


''' Params '''
for loss_fn in ['l1', 'selective']:
    for steps in [1, 10]:
        for epsilon in [1/10, 1/50]:
            threshold = 0.5
            method = 'fgsm' if steps == 1 else 'pgd'

            coco_folder = f'C:/Users/tcilloni/Desktop/coco_cloaked/retinanet_{loss_fn}_{method}_{epsilon:.3f}'
            dataset = load_coco_dataset_in_fiftyone(coco_folder, use_cached=False)
            
            for sample in tqdm(dataset):
                detections = detections_fiftyone_format(sample.filepath, retinanet)
                sample['predictions'] = detections
                sample.save()

            # session.view = dataset.view()
            results = dataset.evaluate_detections('predictions', gt_field='detections', eval_key='eval', compute_mAP=True)
            print(f'{loss_fn} {method} {epsilon:.3f}\t mAP: {results.mAP():.3f}')

skipping


In [13]:
%%script echo skipping
from src.coco_eval import load_coco_dataset_in_fiftyone
from retinanet_utils import *

loss_fn = 'selective'
steps = 1
epsilon = 1/50
threshold = 0.5
method = 'fgsm' if steps == 1 else 'pgd'

coco_folder = f'C:/Users/tcilloni/Desktop/coco_cloaked/retinanet_{loss_fn}_{method}_{epsilon:.3f}'
dataset = load_coco_dataset_in_fiftyone(coco_folder, use_cached=False)

for sample in tqdm(dataset):
    detections = detections_fiftyone_format(sample.filepath, retinanet)
    sample['predictions'] = detections
    sample.save()

# session.view = dataset.view()
results = dataset.evaluate_detections('predictions', gt_field='detections', eval_key='eval', compute_mAP=True)
print(f'{loss_fn} {method} {epsilon:.3f}\t mAP: {results.mAP():.3f}')

skipping


# PASCAL VOC
### Attack

In [14]:
%%script echo skipping
from src.coco_eval import prepare_voc_dataset_folder

for loss_fn in ['l1', 'selective']:
    for steps in [1, 5]:
        for epsilon in [1/10, 1/50]:
            threshold = 0.5
            method = 'fgsm' if steps == 1 else 'pgd'

            dst_folder = f'{loss_fn}_{method}_{epsilon:.3f}'
            src_fnames, dst_fnames = prepare_voc_dataset_folder('data/pascal', f'C:/Users/tcilloni/Desktop/pascal_cloaked/retinanet_{dst_folder}')

            for src, dst in tqdm(list(zip(src_fnames, dst_fnames)), desc = dst_folder):
                image, dims = read_image(src)
                image = image_to_tensor(image)

                cloacked_image = fgsm(image, retinanet, steps, 4.4 * epsilon / steps, threshold, loss_fn)
                cloacked_image = image_to_numpy(cloacked_image)

                save_image(dst, cloacked_image, dims)

skipping


### Evaluation

In [15]:
# %%script echo skipping
from src.coco_eval import load_pascal_dataset_in_fiftyone
from retinanet_utils import *
res = {}


for loss_fn in ['l1', 'selective']:
    for steps in [1, 10]:
        for epsilon in [1/10, 1/50]:
            threshold = 0.5
            method = 'fgsm' if steps == 1 else 'pgd'

            voc_folder = f'C:/Users/tcilloni/Desktop/pascal_cloaked/retinanet_{loss_fn}_{method}_{epsilon:.3f}'
            dataset = load_pascal_dataset_in_fiftyone(voc_folder, use_cached=False)
            
            for sample in tqdm(dataset, desc=voc_folder):
                detections = detections_fiftyone_format(sample.filepath, retinanet)
                sample['predictions'] = detections
                sample.save()

            results = dataset.evaluate_detections('predictions', gt_field='ground_truth', eval_key='eval', method='open-images')
            print(f'{loss_fn} {method} {epsilon:.3f}\t mAP: {results.mAP():.3f}')
            res[f'{loss_fn} {method} {epsilon:.3f}'] = results.mAP()

print(res)

skipping


### Baseline

In [16]:
%%script echo skipping
from src.coco_eval import load_pascal_dataset_in_fiftyone
from retinanet_utils import *

dataset = load_pascal_dataset_in_fiftyone('data/pascal', use_cached=False)

for sample in tqdm(dataset, desc='baseline'):
    detections = detections_fiftyone_format(sample.filepath, retinanet, dataset='pascal')
    sample['predictions'] = detections
    sample.save()

results = dataset.evaluate_detections('predictions', gt_field='ground_truth', eval_key='eval', method='open-images')
print(f'baseline mAP: {results.mAP():.3f}')

 100% |███████████████| 4952/4952 [14.1s elapsed, 0s remaining, 346.9 samples/s]      


baseline:   0%|          | 0/4952 [00:00<?, ?it/s]

Evaluating detections...
 100% |███████████████| 4952/4952 [3.4m elapsed, 0s remaining, 27.9 samples/s]      
baseline mAP: 0.778


In [17]:
import IPython
IPython.Application.instance().kernel.do_shutdown(True) #automatically restarts kernel

{'status': 'ok', 'restart': True}